In [1]:
import asyncio
import os
import time
import random
import pandas as pd
from datetime import datetime, timedelta
import logging
import sys
import powerlaw

from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

import oasis
from oasis import ActionType, EnvAction, SingleAction, Platform
from oasis.social_agent.agent_activation import activation_function
from oasis.social_platform.channel import Channel
from oasis.clock.clock import Clock

/abyss/home/venvs/oasis-env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Parse command line passed arguments
model_name = "qwen"
ip = "127.0.0.1"
port = "8002"
time_steps = 24
subreddit = 6
seed_post = 1
clock_factor = 60

In [3]:
async def main():
    
    model_name = "qwen"
    ip = "127.0.0.1"
    port = "8002"
    time_steps = 24
    subreddit = 6
    seed_post = 1
    clock_factor = 60
    
    # Define the model for the agents
    llm_model = ModelFactory.create(
        model_platform=ModelPlatformType.VLLM,
        model_type=model_name,
	    url=f"http://{ip}:{port}/v1",
    )

    # Define the available actions for the agents
    available_actions = [
        ActionType.LIKE_POST,
        ActionType.DISLIKE_POST,
        ActionType.UNLIKE_POST,
        ActionType.UNDO_DISLIKE_POST,
        # ActionType.CREATE_POST,
        ActionType.CREATE_COMMENT,
        ActionType.LIKE_COMMENT,
        ActionType.DISLIKE_COMMENT,
        ActionType.UNLIKE_COMMENT,
        ActionType.UNDO_DISLIKE_COMMENT,
        ActionType.CREATE_COMMENT_COMMENT,
        # ActionType.SEARCH_POSTS,
        # ActionType.SEARCH_USER,
        # ActionType.TREND,
        # ActionType.REFRESH,
        ActionType.DO_NOTHING,
        # ActionType.FOLLOW,
        # ActionType.MUTE,
        # ActionType.UNMUTE,
    ]

    time_factor = 60/clock_factor
    time_steps = int(time_steps*time_factor)
    db_dir_path = f"/../abyss/home/oasis/oasis-rutschmanna/data/dbs/reddit-sim_{model_name}_pf{subreddit}-{time_steps}h/" # _{time.strftime('%H-%M', time.localtime())}

    if not os.path.exists(db_dir_path):
        os.makedirs(db_dir_path)
        print("Created:", db_dir_path)

    # Define the path to the database
    db_path = f"{db_dir_path}sp{seed_post}.db"

    # Load Oswald et al. sample data and fit powerlaw
    sample_data = pd.read_csv("/../abyss/home/oasis/oswald-et-al_2025/sample_anon.csv")
    interevent_data_seed = pd.read_csv(
        "/../abyss/home/oasis/oasis-rutschmanna/data/reddit/inter_event.csv"
    )["interevent_diff"]
    fit = powerlaw.Fit(
        xmin=interevent_data_seed.min(), 
        xmax=interevent_data_seed.max(), 
        data=interevent_data_seed
    )
    distribution_fit = fit.power_law

    # env = oasis.make(
    #     platform=oasis.DefaultPlatformType.REDDIT
    #     database_path=db_path,
    #     agent_profile_path=f"data/reddit/dp{persona_file}_personas.json",
    #     agent_models=llm_model,
    #     available_actions=available_actions,
    # )

    env = oasis.make(
        platform=Platform(
            db_path=db_path,
            channel=Channel(),
            recsys_type="reddit",
            allow_self_rating=True,
            show_score=True,
            max_rec_post_len=10000,
            sandbox_clock=Clock(clock_factor),
            start_time=datetime(2025, 7, 1, 12, 00, 00) + timedelta(hours=24*(seed_post-1)),
        ),
        database_path=db_path,
        agent_profile_path=f"/../abyss/home/oasis/oasis-rutschmanna/data/reddit/seed_personas_subreddit_{subreddit}.json",
        agent_models=llm_model,
        available_actions=available_actions,
        semaphore=16,
    )

    seed_posts = {
        1: "The US should condemn Israel’s military actions in Gaza as acts of genocide and impose full sanctions.",
        2: "Prostitution should be illegal.",
        3: "Things like gender-neutral language and stating pronouns are silly issues.",
        4: "The death penalty should be reestablished US-wide.",
        5: "The US should provide financial and military aid to Ukraine.",
        6: "We need stricter gun control laws.",
        7: "Social media is a threat to democracy.",
        8: "Immigration should be regulated more strictly.",
        9: "Fur clothing should be banned.",
        10: "Police officers should wear body cameras.",
        11: "Climate change is one of the greatest threats to humanity.",
        12: "Employers should mandate vaccination.",
        13: "The government should not be responsible for providing universal health care.",
        14: "The government should not forgive student loan debt.",
        15: "Artificial intelligence should replace humans where possible.",
        16: "There should only be vegetarian food in cantines.",
        17: "A universal basic income would kill the economy.",
        18: "The federal minimum wage should be increased.",
        19: "The government should not invest in renewable energy.",
        20: "Airbnb should be banned in cities."
    }

    # Run the environment
    await env.reset()

    action_1 = SingleAction(
            agent_id=0,
            action=ActionType.CREATE_POST,
            args={
                "content": (
                    f"{seed_posts[seed_post]}\n"
                    "Please discuss! This statement serves as starting point for today’s discussion."
                    # "It neither reflects the opinion of the researchers nor a political "
                    # "position of the research institution."
                    )
                }
            )

    activated_agents = [0]
    env_action_1 = EnvAction(
        activate_agents=activated_agents,
        intervention=[action_1]
    )

    # Perform the actions
    await env.step(env_action_1)
    
    for _ in range(time_steps):
        activated_agents = activation_function(
            sample_data.set_index("ParticipantID"),
            db_path,
            env.platform,
            mapping_type="comments",
            distribution_fit=distribution_fit,
            recurring_activation_prob_modifier=0.5,
        )

        if len(activated_agents) > 0:
            env_action_empty = EnvAction(
                activate_agents=activated_agents
            )
            await env.step(env_action_empty)
        else:
            sleep = random.randint(
                int((time_factor*0.8)*60),
                int(time_factor*60)
            )
            time.sleep(sleep)

    # Close the environment
    await env.close()

In [6]:
# await main()